In [1]:
'''
Differenciate pictures of chihuahua from muffins
Data from: https://github.com/rcgc/chihuahua-muffin
'''

'\nDifferenciate pictures of chihuahua from muffins\nData from: https://github.com/rcgc/chihuahua-muffin\n'

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

In [3]:
# train_df = pd.DataFrame(columns=["img_name","label"])
# train_df["img_name"] = os.listdir(path_train)
# for idx, i in enumerate(os.listdir(path_train)):
#     if "cat" in i:
#         train_df["label"][idx] = 0
#     if "dog" in i:
#         train_df["label"][idx] = 1

# train_df.to_csv (r'train_csv.csv', index = False, header=True)
#path = "/Users/poho/Documents/Data/chic_muff"
path_to_data = "/Users/poho/Documents/Data/pos_dog_muffin"
#create an empty dataframe for annotation df
# annotation_df = pd.DataFrame(columns = ['img_name', 'label'])
# #list all directories
# annotation_df['img_name'] = os.listdir(path_to_data)
# #label files with chihuahua as 0 and muffin as 1
# for idx, i in enumerate(os.listdir(path_to_data)):
#     if 'chihuahua' in i:
#         annotation_df['label'][idx] = 0
#     if 'muffin' in i:
#         annotation_df['label'][idx] = 1
        
# annotation_df.to_csv(r'annotations.csv', index = False, header = True)

In [4]:
#https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
# class ChiMuffin(Dataset):
#     def __init__(self, annotation_df, img_dir, transform= None, target_transform = None):
#         #storing image labels(ground truth)
#         self.img_labels = pd.read_csv(annotation_df)
#         #storing image directory
#         self.img_dir = img_dir
#         self.transform = transform
#         self.target_transform = target_transform
        
#     def __len__(self):
#         return len(self.imag_labels)
    
#     def __getitem__(self, idx):
#         #get the path to images
#         img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
#         #put the image in the memory
#         image = read_image(img_path)
#         #get the label
#         label = self.img_labels.iloc[idx, 1]
#         #perform trasnformation
#         if self.transform:
#             image = self.transform(image)
#         if self.target_transform:
#             label = self.target_transform(label)
#         return image, label
        

In [5]:
#https://debuggercafe.com/pytorch-imagefolder-for-training-cnn-models/
#transform images to the same size(pixel 400*400)
import torchvision.transforms as transforms
train_transform = transforms.Compose([
    transforms.Resize((400,400)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.RandomRotation(degrees=(30, 70)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5])
])

valid_transform = transforms.Compose([
    transforms.Resize((400, 400)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

In [6]:
new_path = "/Users/poho/Documents/Data/chic_muff"
ChiMuffin_dataset = datasets.ImageFolder(root=new_path,
                                           transform=train_transform)

dataset_loader = torch.utils.data.DataLoader(ChiMuffin_dataset,
                                             batch_size=4, shuffle=True,
                                             num_workers=4)

new_path_val = "/Users/poho/Documents/Data/chic_muff_val"
ChiMuffin_dataset_val = datasets.ImageFolder(root=new_path_val,
                                           transform=valid_transform)

dataset_loader_val = torch.utils.data.DataLoader(ChiMuffin_dataset_val,
                                             batch_size=4, shuffle=True,
                                             num_workers=4)


In [7]:
#checking number of training dataset (80% training data 20% validation)
# len(dataset_loader)*4

In [11]:
import torch.nn as nn
import torch.nn.functional as F
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.conv4 = nn.Conv2d(128, 256, 5)
        
        self.fc1 = nn.Linear(256, 2)
        
        self.pool = nn.MaxPool2d(2, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        bs, _, _, _ = x.shape
        x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        x = self.fc1(x)
        return x

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm


# learning_parameters 
lr = 1e-3
epochs = 20
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}\n")
model = CNNModel().to(device)
print(model)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)
# loss function
criterion = nn.CrossEntropyLoss()

Computation device: cpu

CNNModel(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=2, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [12]:
# def train(epochs, model, optimizer, criterion, train_iterator, valid_iterator):
#     for epoch in range(1, epochs + 1):
#         training_loss = 0.0 #0.0 because you want it to be float
#         valid_loss = 0.0
#         model.train()
#         for batch_idx, batch in enumerate(train_iterator):
#             opt.zero_grad() #initialize our gradient to zero
#             preidct = model(batch.tweet)
        
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [16]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataset_loader, model, criterion, optimizer)
    test(dataset_loader_val, model, criterion)
print("Done!")

Epoch 1
-------------------------------
loss: 0.687502  [    4/ 1118]
loss: 0.572710  [  404/ 1118]
loss: 0.365624  [  804/ 1118]
Test Error: 
 Accuracy: 75.9%, Avg loss: 0.601196 

Epoch 2
-------------------------------
loss: 0.401916  [    4/ 1118]
loss: 0.604441  [  404/ 1118]
loss: 0.214441  [  804/ 1118]
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.371201 

Epoch 3
-------------------------------
loss: 0.342858  [    4/ 1118]
loss: 0.488919  [  404/ 1118]
loss: 0.185936  [  804/ 1118]
Test Error: 
 Accuracy: 86.9%, Avg loss: 0.355391 

Epoch 4
-------------------------------
loss: 0.417994  [    4/ 1118]
loss: 0.197935  [  404/ 1118]
loss: 0.440636  [  804/ 1118]
Test Error: 
 Accuracy: 86.5%, Avg loss: 0.297877 

Epoch 5
-------------------------------
loss: 0.121506  [    4/ 1118]
loss: 0.266073  [  404/ 1118]
loss: 1.426667  [  804/ 1118]
Test Error: 
 Accuracy: 80.1%, Avg loss: 0.487669 

Done!
